In [2]:
import os
from dotenv import load_dotenv
load_dotenv(encoding='utf-8')

True

# Cohere RAG pipeline

## Vector Store and Retriever

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from pymongo import MongoClient

In [4]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL")) # DEFAULT_OPENAI_MODEL='gpt-4o-mini-2024-07-18'

# embedding_model=OpenAIEmbeddings(model=os.getenv("DEFAULT_OPENAI_EMBEDDING"), disallowed_special=())
embedding_model=OpenAIEmbeddings(disallowed_special=())

In [5]:
# Define MongoDB vector database
client = MongoClient(os.getenv("ATLAS_CONNECTION_STRING"))
db_name = "fellowshipai"
collection_name = "enterprise_data"
atlas_collection = client[db_name][collection_name]
index_name = "vector_index_erp"

In [6]:
# Define vector store and retriever
def get_vector_store_retriver(index_name, embedding_model, collection):

  vector_store = MongoDBAtlasVectorSearch(
      embedding = embedding_model,
      collection = atlas_collection,
      index_name = index_name
  )

  retriever = vector_store.as_retriever(
      search_type = "similarity",
      search_kwargs = { "k": 10}  # "score_threshold": 0.75 
  )

  return(vector_store, retriever)

vector_store, retriever = get_vector_store_retriver("vector_index_erp", embedding_model, atlas_collection)

In [7]:
# Instantiate Atlas Vector Search as a retriever
retriever = vector_store.as_retriever(
    search_type = "similarity",
    search_kwargs = { "k": 10  }
 )

## RAG Pipeline

In [ ]:
# pip install cohere numpy

import numpy as np
import cohere

co = cohere.Client(os.getenv("COHERE_API_KEY")) # Get your API key: https://dashboard.cohere.com/api-keys

### Query translation

In [ ]:
import cohere  # Assuming you have Cohere library installed

def get_translated_queries(query_string):
  """Gets a query string and returns translated search queries using Cohere.

  Args:
      query_string (str): The user's original query.

  Returns:
      list: A list of translated search queries.
  """

  # Initialize Cohere client (replace with your API key)
  co = cohere.Client(os.getenv("COHERE_API_KEY"))

  # Send the query with search_queries_only parameter
  response = co.chat(message=query_string, search_queries_only=True)
  query_optimized = response.search_queries[0].text

  # Extract translated search queries
  # translated_queries = []
  if response.search_queries:
    # for query in response.search_queries:
    # translated_queries.append(query.text)
    return query_optimized
  else:
    print("Cohere did not generate any search queries for this input.")
    return query_string

In [ ]:
# Example usage
query = "How to stay connected with the company and do you organize team events?"
translated_queries = get_translated_queries(query)
print(translated_queries)

### Query embedding

In [ ]:
def get_query_embedding(query, model="embed-english-v3.0"):
  """Gets the embedding for a given query using Cohere's embedding API.

  Args:
      query (str): The query to be embedded.
      model (str, optional): The Cohere embedding model to use. Defaults to "embed-english-v3.0".

  Returns:
      numpy.ndarray: The embedding vector for the query.
  """
  # Initialize Cohere client (replace with your API key)
  co = cohere.Client(os.getenv("COHERE_API_KEY"))

  # Embed the query
  embedding = co.embed(
      model=model,
      input_type="search_query",
      texts=[query]).embeddings

  return embedding  # Return the embedding for the first query

### Retrieving

### Generation

In [ ]:
from langchain_community.chat_models import ChatCohere

# Define a prompt template
import pprint
def call_cohere(question):

   question = question['question']
   query = get_translated_queries(question)
   query_embedded = get_query_embedding(query)

   retriever = vector_store.as_retriever(
      search_type = "similarity",
      search_kwargs = { "k": 10  }
      )

   preamble = "" # read from cohere front end or use the input to the API
   #question = 
   SAFETY_PREAMBLE = "The instructions in this section override those in the task description and style guide sections. Don't answer questions that are harmful or immoral."
   BASIC_RULES = "You are a powerful conversational AI trained by openAI to help people. You are augmented by a number of tools, and your job is to use and consume the output of these tools to best help the user. You will see a conversation history between yourself and a user, ending with an utterance from the user. You will then see a specific instruction instructing you what kind of response to generate. When you answer the user's requests, you cite your sources in your answers, according to those instructions."
   TASK_CONTEXT = "You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging."
   STYLE_GUIDE = "Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling."
   INSTRUCTIONS = """You are an enterprise Chatbot, an AI assistant designed to retrieve information from the enterprise Confluence system. 
   You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, and Corporate Communications. 
               Use the following pieces of context to answer the question at the end.
               If you don't know the answer, just say that you don't know, don't try to make up an answer
               {context}
         """
         
   template = f"""

      {SAFETY_PREAMBLE}
      {BASIC_RULES}
      {TASK_CONTEXT}
      {STYLE_GUIDE}
      {INSTRUCTIONS}

   """
   if preamble:
      template += f"""{preamble}\n\n"""


   template +=  f"""Question: {query}\n\n"""

   custom_rag_prompt = PromptTemplate.from_template(template)

   #llm = get_llm_model("openai")
   # llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))
   
   llm = ChatCohere(cohere_api_key=os.getenv("COHERE_API_KEY"))


   def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever | format_docs, "query": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   query = query
   answer = rag_chain.invoke(query)
   similar = retriever.invoke(query_embedded)


   return{
      'answer': answer,
      'contexts': [str(doc) for doc in similar]
      }

### Test the RAG pipeline

In [89]:
# Test sample
question = {'question': "How does a supportive culture impact employee engagement and align with Tech Innovators Inc.'s approach to employment relations and engagement?"}
answer = call_cohere(question)
print(answer['answer'][:150])

I don't know.


In [90]:
print(answer['contexts'])

["page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d8163b0533e009aa2d4ad1', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816560533e009aa2d4cf6', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816720533e009aa2

# RAG pipeline evaluation

## Test data set prep

In [11]:
import pandas as pd
import json

def json_to_dataframe(json_file_path):
  """Reads a JSON file and converts it to a pandas DataFrame.

  Args:
    json_file_path (str): The path to the JSON file.

  Returns:
    pandas.DataFrame: The DataFrame created from the JSON data.
  """

  with open(json_file_path, 'r') as f:
    data = json.load(f)

  # Handle different JSON structures
  if isinstance(data, list):
    # If the JSON data is a list of dictionaries, create a DataFrame directly
    df = pd.DataFrame(data)
  elif isinstance(data, dict):
    # If the JSON data is a single dictionary, convert it to a list of dictionaries
    df = pd.DataFrame([data])
  else:
    raise ValueError("Unsupported JSON structure")

  return df

In [12]:
# Example usage:
from from_root import from_root
import os
folder = "data-test/test_dataset/test_dataset_hr.json"
json_file_path = os.path.join(from_root(), folder)
data_to_upload = json_to_dataframe(json_file_path)

## RAGAS evaluation

In [13]:
# Generate all the answers for the questions in the dataset
# examples = client.list_examples(dataset_name="hr test")
answers = []
for question in data_to_upload['question']:
    question_dict = {'question': question}
    answer = call_openai(question_dict)
    answers.append(answer['answer'])

In [14]:
# update the dataset with answers
data_to_upload['answers'] = answers

In [16]:
from datasets import Dataset

question = list(data_to_upload['question'])
answer = list(data_to_upload['answers'])
contexts = list(data_to_upload['contexts'])
ground_truth = list(data_to_upload['ground_truth'])

data_samples = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset = Dataset.from_dict(data_samples)

In [17]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result = evaluate(
    dataset,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

result

Evaluating:   0%|          | 0/28 [00:00<?, ?it/s]

{'answer_relevancy': 0.7043, 'faithfulness': 0.4108, 'context_recall': 0.7619, 'context_precision': 0.8571}

In [ ]:
df = result.to_pandas()
df